In [19]:
#Import libraries for simulation
import tensorflow as tf
import numpy as np

# Import libraries for alg manipulation
from numpy.linalg import inv
import scipy as sp
import scipy.sparse

#Imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display


In [20]:
def DisplayArray(a, fmt='jpeg', rng=[0,1]):
  """Display an array as a picture."""
  a = (a - rng[0])/float(rng[1] - rng[0])*255
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))


In [21]:
sess = tf.InteractiveSession()


/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [22]:
def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

def simple_conv(x, k):
  """A simplified 2D convolution operation"""
  x = tf.expand_dims(tf.expand_dims(x, 0), -1)
  y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
  return y[0, :, :, 0]

def laplace(x):
  """Compute the 2D laplacian of an array"""
  laplace_k = make_kernel([[0.5, 1.0, 0.5],
                           [1.0, -6., 1.0],
                           [0.5, 1.0, 0.5]])
  return simple_conv(x, laplace_k)


In [23]:
def get_static_sol(filename):
    file = open(filename, 'r')
    X = []
    Y = []
    for line in file:
        aux1, aux2 = line.split()
        X.append(float(aux1))
        Y.append(float(aux2))
    return X, Y

def get_next_Psi(Psi, A, b):
    invA = inv(A)
    return np.dot(invA, b)

def invert(A):
    aux = []
    for i in range(0,len(A)):
        aux.append(A[len(A)-1-i])
    return aux


#Adquirir condiciones de borde
print("retrieving files")
x_cp, numCp = np.asarray(get_static_sol('../ch2/results-steadystate/cp-num-r5e-08.txt'))
x_cm, numCm = np.asarray(get_static_sol('../ch2/results-steadystate/cm-num-r5e-08.txt'))
x_psi, numPsi = np.asarray(get_static_sol('../ch2/results-steadystate/potential-num-r5e-08.txt'))


Psi0 = numPsi[0]
dx = x_cp[1]-x_cp[0]
Cb = 0.1

#Inicializar los valores
N = 1000
M = len(x_cp)

Cp = np.zeros([N,M])
Cm = np.zeros([N,M])
Psi  = np.zeros([N,M])

print("setting up border condition")
for j in range(1,M):
    #Cp[0, j] = numCp[j]
    #Cm[0, j] = numCm[j]
    #Psi[0, j] = numPsi[j]
    Cp[0, j] = Cb
    Cm[0, j] = Cb
    Psi[0, j] = 0

    
for i in range(0,N):
    Cp[i,M-1] = Cb
    Cm[i,M-1] = Cb
    Psi[i,0] = Psi0
    Psi[i,M-1] = 0


#hasta aca asumo que conocemos las condiciones de borde.
#definicion de constantes
z = 2
e = 1.60217662E-19
k = 1.38064852E-23
T = 300
Na = 6.02E23
Fa = Na * e
R = Na * k
coef = z * e / (k * T)
V_0 = -coef * 0.15

epsilon = 80.9 * 8.85418782E-12
k = np.sqrt(0.1 * (z * Fa) ** 2 / (R * T * epsilon))
dt = 1
k = dx * k
Dp = 0.00005
Dm = 0.00005
rp = dt * Dp / dx ** 2
rm = dt * Dm / dx ** 2

#r = k * 5E-8
r = 5E-8


#Aca definimos el sistema que resuleve para cada iteracion del potencial.
# La matriz de coeficientes es M x M donde M es el numero de pasos en el eje x
print("creating coefficient matrix")
a = np.ones(M)
b1 = -2 * np.ones(M)
c = np.ones(M)
positions = [-1, 0, 1]
A = sp.sparse.spdiags(np.array([a, b1, c]), positions, M, M).todense()
Ainv = np.asarray(np.linalg.inv(A))

print("done creating coefficient matrix")
print("Setting up resulting vector")
b = np.zeros([N,M])

print("done...")

retrieving files
setting up border condition
creating coefficient matrix
done creating coefficient matrix
Setting up resulting vector
done...


In [24]:
# Initial Conditions -- some rain drops hit a pond

# Set everything to zero
Psi_init = np.zeros([N, N], dtype=np.float32)
Cpt_init = np.zeros([N, N], dtype=np.float32)
Cmt_init = np.zeros([N, N], dtype=np.float32)



In [25]:
# Parameters:
# eps -- time resolution
# damping -- wave damping
eps = tf.placeholder(tf.float32, shape=())
damping = tf.placeholder(tf.float32, shape=())

# Create variables for simulation state
Psi  = tf.Variable(Psi_init)
Cpt = tf.Variable(Cpt_init)
Cmt = tf.Variable(Cmt_init)

# Discretized PDE update rules
Psi_ = Psi + eps * (Cpt - Cmt)
Cpt_ = Cpt + eps * (laplace(Cp))
Cmt_ = Cmt + eps * (laplace(Cm))

# Operation to update the state
step = tf.group(
  U.assign(U_),
  Ut.assign(Ut_))


TypeError: Input 'filter' of 'DepthwiseConv2dNative' Op has type float32 that does not match type float64 of argument 'input'.

In [26]:
# Initialize state to initial conditions
tf.global_variables_initializer().run()

# Run 1000 steps of PDE
for i in range(1000):
  # Step simulation
  step.run({eps: 0.03, damping: 0.04})
  DisplayArray(U.eval(), rng=[-0.1, 0.1])


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  File "/anaconda3/lib/python3.6/site-packages/tornad